In [ ]:
import numpy as np
import pandas as pd

#READ FILES
bill_details_df = pd.read_excel('monthly_bd.xlsx')
ppm_df= pd.read_excel('VendingDetails.xlsx') #read billing details file
adjustments_df= pd.read_excel('June Adjustment.xlsx') #read billing details file
payments_df= pd.read_excel('Postpaid_payments.xlsx') #read billing details file


#Change ff columns to caps (for presentation)
ppm_df.rename(columns = {'TariffCalssName':'TariffClassName'}, inplace = True) 

bill_details_df['BusinessUnitName']= bill_details_df['BusinessUnitName'].str.upper()
bill_details_df['ServiceUnitName']= bill_details_df['ServiceUnitName'].str.upper()
bill_details_df['TariffClassName']= bill_details_df['TariffClassName'].str.upper()
bill_details_df['CustomerType']= bill_details_df['CustomerType'].str.upper()

adjustments_df['BusinessUnitName']= adjustments_df['BusinessUnitName'].str.upper()
adjustments_df['SERVICEUNITNAME']= adjustments_df['SERVICEUNITNAME'].str.upper()
adjustments_df['TARIFFCLASSNAME']= adjustments_df['TARIFFCLASSNAME'].str.upper()
adjustments_df['CustomerType']= adjustments_df['CustomerType'].str.upper()

payments_df['BusinessUnitName']= payments_df['BusinessUnitName'].str.upper()
payments_df['ServiceUnitName']= payments_df['ServiceUnitName'].str.upper()
payments_df['TariffClassName']= payments_df['TariffClassName'].str.upper()
payments_df['CustomerType']= payments_df['CustomerType'].str.upper()

ppm_df['BusinessUnitName']= ppm_df['BusinessUnitName'].str.upper()
ppm_df['ServiceUnitName']= ppm_df['ServiceUnitName'].str.upper()
ppm_df['TariffClassName']= ppm_df['TariffClassName'].str.upper()
ppm_df['CustomerType']= ppm_df['CustomerType'].str.upper()

#create sub df from read files
sub_bill_details_df= bill_details_df[['BusinessUnitName','ServiceUnitName','GlobalAccountNumber', 'TariffClassName', 'CustomerType', 'DisplayCode', 'EBMSConsumption', 'EnergyCharges', ' BilledAmount ', 
                                      'MeterNumber', 'PreviousBalance', 'OutstandingAmount', 'TariffRates']]
sub_ppm_df= ppm_df[['ServiceUnitName','BusinessUnitName','TariffClassName', 'Units', 'CostOfUnits', 'Actual Payment Collection', 'AccountNo', 'TariffRate',
                    'VendingNo','PaymentAmount','CustomerType']]
sub_payment_df= payments_df[['BusinessUnitName','ServiceUnitName','TariffClassName','CustomerType','AccountNo', 'PaidAmount']]
sub_adjustment_df= adjustments_df[['BusinessUnitName','SERVICEUNITNAME','TARIFFCLASSNAME','CustomerType','AccountNo', 'AmountEffected']]


#fix PPM. payment and adj sub df
sub_ppm_df['CustomerType'] = sub_ppm_df['CustomerType'].replace(['NON MD','MD', 'PRIME'],'PPM')
sub_ppm_df['DisplayCode'] = "ACTIVE"
#
sub_ppm_df.rename(columns = {'Units':'EBMSConsumption'}, inplace = True)
sub_ppm_df.rename(columns = {'CostOfUnits':'EnergyCharges'}, inplace = True)
sub_ppm_df.rename(columns = {'Actual Payment Collection':' BilledAmount '}, inplace = True)
sub_ppm_df.rename(columns = {'AccountNo':'MeterNumber'}, inplace = True)
sub_ppm_df.rename(columns = {'TariffRate':'TariffRates'}, inplace = True)
sub_ppm_df.rename(columns = {'VendingNo':'GlobalAccountNumber'}, inplace = True)
sub_ppm_df.rename(columns = {'PaymentAmount':'PaidAmount'}, inplace = True)
#
sub_payment_df.rename(columns = {'AccountNo':'PmtAccountNumber'}, inplace = True) # RENAME column headers
#
sub_adjustment_df.rename(columns = {'AccountNo':'AdjustmentAccountNumber'}, inplace = True) # RENAME column headers
sub_adjustment_df.rename(columns = {'SERVICEUNITNAME':'ServiceUnitName', 'TARIFFCLASSNAME': 'TariffClassName'}, inplace = True) 

#fix bd meter number column
sub_bill_details_df['MeterNumber'] = sub_bill_details_df['MeterNumber'].apply(lambda x: "METERED" if not pd.isnull(x) else 'NOMETER')

#create extra columns for agg
sub_bill_details_df['ActiveCustomers']= 0
sub_bill_details_df['Inactive']= 0
sub_bill_details_df['Total']= 0
sub_bill_details_df['Metered']= 0
sub_bill_details_df['Unmetered']= 0
sub_bill_details_df['DirectCustomers']= 0
sub_bill_details_df['ReadCustomers']= 0
sub_bill_details_df['AverageCustomers']= 0
sub_bill_details_df['CustomersonMinimum']= 0

#edit the created columns for agg sum
sub_bill_details_df.loc[sub_bill_details_df['DisplayCode'].notna(), 'ActiveCustomers'] = 1
sub_bill_details_df.loc[sub_bill_details_df['DisplayCode'].isna(), 'Inactive'] = 1
sub_bill_details_df.loc[sub_bill_details_df['GlobalAccountNumber'].notna(), 'Total'] = 1
sub_bill_details_df.loc[sub_bill_details_df['MeterNumber'] == 'METERED', 'Metered'] = 1
sub_bill_details_df.loc[sub_bill_details_df['MeterNumber'] == 'NOMETER', 'Unmetered'] = 1
sub_bill_details_df.loc[sub_bill_details_df['DisplayCode'] == 'D', 'DirectCustomers'] = 1
sub_bill_details_df.loc[sub_bill_details_df['DisplayCode'] == 'R', 'ReadCustomers'] = 1
sub_bill_details_df.loc[sub_bill_details_df['DisplayCode'] == 'A', 'AverageCustomers'] = 1
sub_bill_details_df.loc[sub_bill_details_df['DisplayCode'] == 'M', 'CustomersonMinimum'] = 1
##create extra columns for agg
sub_bill_details_df['Total_']= sub_bill_details_df['Total']
sub_bill_details_df['No_of_Bills']= sub_bill_details_df['ActiveCustomers']



#concate pmt, bd and adj sub dfs
sub_bd_pmt_combined_2 = pd.concat([sub_bill_details_df, sub_payment_df],axis=0)
pmt_adj_bd_combined_2= pd.concat([sub_bd_pmt_combined_2, sub_adjustment_df],axis=0)

#create tariff description column
pmt_adj_bd_combined_2['TariffDesc']= pmt_adj_bd_combined_2['TariffClassName']

#get unique tariff classes
unique_tariff_class_name= (pmt_adj_bd_combined_2['TariffClassName']).sort_values(ascending=True).unique() #get unique tariff names and Sort them alphabetically
unique_tariff_class_name_copy= unique_tariff_class_name.copy()# create a copy to get Tariff Description

#create tariff description from tariffclassname
pmt_adj_bd_combined_2['TariffDesc'] = list(map(lambda x: x.replace('(A1S)','').replace('(A1T)','').replace('(C1S)','')
                                         .replace('(C1T)','').replace('(C2)','').replace('(D1T)','')
                                         .replace('(D2)','').replace('(R2S)','').replace('(R2T)','')
                                         .replace('(R3)','').replace('(R4)','').replace('(A2)','')
                                         .replace('(C3)','').replace('(R1)','').replace('(D1S)','')
                                         .replace('(S1)',''),pmt_adj_bd_combined_2['TariffDesc']))

#agg for postpaid combined bs report (sheet 2)--> post_billing_summary_by_tariff
pd.options.display.float_format = '{:.2f}'.format
post_billing_summary_by_tariff = pmt_adj_bd_combined_2.groupby(['BusinessUnitName', 'TariffDesc', 'TariffClassName'])\
    .agg({'ActiveCustomers':'sum',  'Inactive':'sum', 'Total': 'sum', 'EBMSConsumption':'sum', 'EnergyCharges':'sum', ' BilledAmount ': 'sum','Metered':'sum',
         'Unmetered':'sum','Total_':'sum', 'DirectCustomers':'sum','AverageCustomers':'sum','CustomersonMinimum':'sum','ReadCustomers':'sum', 'No_of_Bills':'sum', 
         'PmtAccountNumber':'count', 'PaidAmount':'sum', 'PreviousBalance':'sum', 'OutstandingAmount':'sum', 'AmountEffected':'sum','TariffRates': 'mean'}).round(2)
#print(post_billing_summary_by_tariff)

# repeat for PPM sub df
sub_ppm_df_copy=sub_ppm_df.copy()

sub_ppm_df_copy['Inactive']= 0
sub_ppm_df_copy['Total']= 0
sub_ppm_df_copy['Metered']= 0
sub_ppm_df_copy['Unmetered']= 0
sub_ppm_df_copy['DirectCustomers']= 0
sub_ppm_df_copy['ReadCustomers']= 0
sub_ppm_df_copy['AverageCustomers']= 0
sub_ppm_df_copy['CustomersonMinimum']= 0
sub_ppm_df_copy['AmountEffected']= 0
sub_ppm_df_copy['OutstandingAmount']= 0
sub_ppm_df_copy['PreviousBalance']= 0

sub_ppm_df_copy.rename(columns = {'GlobalAccountNumber':'PPM_AccountNumber'}, inplace = True)

sub_ppm_df_copy.loc[sub_ppm_df_copy['DisplayCode'].notna(), 'ActiveCustomers'] = 1
sub_ppm_df_copy['DisplayCode']= sub_ppm_df_copy['DisplayCode'].fillna('INACTIVE') #check
sub_ppm_df_copy.loc[sub_ppm_df_copy['DisplayCode'] == 'INACTIVE', 'Inactive'] = 1
sub_ppm_df_copy.loc[sub_ppm_df_copy['PPM_AccountNumber'].notna(), 'Total'] = 1
sub_ppm_df_copy.loc[sub_ppm_df_copy['MeterNumber'] == 'METERED', 'Metered'] = 1
sub_ppm_df_copy.loc[sub_ppm_df_copy['MeterNumber'] == 'NOMETER', 'Unmetered'] = 1
sub_ppm_df_copy.loc[sub_ppm_df_copy['DisplayCode'] == 'D', 'DirectCustomers'] = 1
sub_ppm_df_copy.loc[sub_ppm_df_copy['DisplayCode'] == 'ACTIVE', 'ReadCustomers'] = 1
sub_ppm_df_copy.loc[sub_ppm_df_copy['DisplayCode'] == 'A', 'AverageCustomers'] = 1
sub_ppm_df_copy.loc[sub_ppm_df_copy['DisplayCode'] == 'M', 'CustomersonMinimum'] = 1

sub_ppm_df_copy['Total_']= sub_ppm_df_copy['Total']
sub_ppm_df_copy['No_of_Bills']= sub_ppm_df_copy['ActiveCustomers']
sub_ppm_df_copy['TariffDesc']= sub_ppm_df['TariffClassName']

sub_ppm_df['MeterNumber'] = sub_ppm_df['MeterNumber'].apply(lambda x: "METERED" if not pd.isnull(x) else 'NOMETER')

sub_ppm_df_copy['TariffDesc'] = list(map(lambda x: x.replace('(A1S)','').replace('(A1T)','').replace('(C1S)','')
                                         .replace('(C1T)','').replace('(C2)','').replace('(D1T)','')
                                         .replace('(D2)','').replace('(R2S)','').replace('(R2T)','')
                                         .replace('(R3)','').replace('(R4)','').replace('(A2)','')
                                         .replace('(C3)','').replace('(R1)','').replace('(D1S)','')
                                         .replace('(S1)',''),sub_ppm_df_copy['TariffDesc']))


#agg for prepaid combined bs report (sheet )--> ppm_billing_summary_by_tariff
ppm_billing_summary_by_tariff = sub_ppm_df_copy.groupby(['BusinessUnitName', 'TariffDesc', 'TariffClassName'])\
    .agg({'ActiveCustomers':'sum',  'Inactive':'sum', 'Total': 'sum', 'EBMSConsumption':'sum', 'EnergyCharges':'sum', ' BilledAmount ': 'sum','Metered':'sum',
         'Unmetered':'sum','Total_':'sum', 'DirectCustomers':'sum','AverageCustomers':'sum','CustomersonMinimum':'sum','ReadCustomers':'sum', 'No_of_Bills':'sum', 
         'PPM_AccountNumber':'count', 'PaidAmount':'sum', 'PreviousBalance':'sum', 'OutstandingAmount':'sum', 'AmountEffected':'sum', 'TariffRates': 'mean'}).round(2)
#print(ppm_billing_summary_by_tariff)

#begining of sheet 1
sub_ppm_df_copy['PmtAccountNumber']= sub_ppm_df_copy['PPM_AccountNumber']

pmt_adj_bd_ppm_combined= pd.concat([pmt_adj_bd_combined_2, sub_ppm_df_copy],axis=0)
#agg for prepaid combined bs report (sheet )--> bs_by_customer_type
bs_by_customer_type = pmt_adj_bd_ppm_combined.groupby(['BusinessUnitName', 'ServiceUnitName', 'CustomerType'])\
    .agg({'ActiveCustomers':'sum', 'Inactive':'sum', 'Total':'sum',  'EBMSConsumption':'sum', 'EnergyCharges':'sum',
          ' BilledAmount ': 'sum', 'Metered':'sum', 'Unmetered':'sum', 'Total_':'sum','DirectCustomers':'sum','AverageCustomers':'sum'
         ,'CustomersonMinimum':'sum','ReadCustomers':'sum', 'No_of_Bills':'sum','PmtAccountNumber':'count', 'PaidAmount':'sum', 'PreviousBalance':'sum'
         , 'OutstandingAmount':'sum', 'AmountEffected':'sum'}).round(2)
#print(bs_by_customer_type)
#create dictionary for renaming of columns
dict = {'ActiveCustomers': 'Active',
        'Inactive': 'In_active',
        'EBMSConsumption': 'Energy_Billed',
       'EnergyCharges': 'Revenue_Billed',
       'DirectCustomers': 'Direct',
       'AverageCustomers': 'Estimated',
       'CustomersonMinimum': 'Min_FC',
        'ReadCustomers': 'Read',
        'PmtAccountNumber': 'Response',
        'PPM_AccountNumber': 'Response',
        'TotalPayment': 'Payment',
       'PreviousBalance': 'Opening_Balance',
       'OutstandingAmount': 'Closing_Balance',
       'AmountEffected': 'Adjustment',
       'PaidAmount': 'Payment',}

#get list and count of BUs
list_of_bu= bill_details_df['BusinessUnitName'].dropna().sort_values(ascending=True).unique()
list_of_bu
number_of_bu= len(list_of_bu)

#Creating a df for sheet 1 from groupby file
consolidated_billing_summary_by_customer_type= bs_by_customer_type.reset_index()
consolidated_billing_summary_by_customer_type.rename(columns=dict, inplace=True)

#Creating a df for sheet 2 from groupby file
postpaid_billing_summary_by_tariff= post_billing_summary_by_tariff.reset_index()
postpaid_billing_summary_by_tariff.rename(columns=dict, inplace=True)

#Creating a df for sheet 3 from groupby file
cons_ppm_billing_summary_by_tariff= ppm_billing_summary_by_tariff.reset_index()
cons_ppm_billing_summary_by_tariff.rename(columns=dict, inplace=True)

#This generates report for indiovidual BUs 
for bu in range(number_of_bu):
    buName= list_of_bu[bu]
    #consolidated_first_sheet= billing_summary_by_customer_type_df[billing_summary_by_customer_type_df['BusinessUnitName']==list_of_bu[buName]]
    
    consolidated_first_sheet= consolidated_billing_summary_by_customer_type[consolidated_billing_summary_by_customer_type['BusinessUnitName']==buName]
    
    consolidated_second_sheet= postpaid_billing_summary_by_tariff[postpaid_billing_summary_by_tariff['BusinessUnitName']==buName]
    consolidated_third_sheet= cons_ppm_billing_summary_by_tariff[cons_ppm_billing_summary_by_tariff['BusinessUnitName']==buName]
    
    with pd.ExcelWriter(buName + ' Billing_Status.xlsx') as writer:  
            consolidated_first_sheet.to_excel(writer, sheet_name='BILLING_SUMMARY_BY_CUSTOMER_TYP', index=False, index_label='SNO')
            consolidated_second_sheet.to_excel(writer, sheet_name='POST_BILLING_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
            consolidated_third_sheet.to_excel(writer, sheet_name='PPM_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
#print(bu_only_report_df)

# THIS GENERATES CONSOLIDATED FILE
with pd.ExcelWriter('Consolidated Billing_Status.xlsx') as writer:  
            consolidated_billing_summary_by_customer_type.to_excel(writer, sheet_name='BILLING_SUMMARY_BY_CUSTOMER_TYP', index=False, index_label='SNO')
            postpaid_billing_summary_by_tariff.to_excel(writer, sheet_name='POST_BILLING_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
            cons_ppm_billing_summary_by_tariff.to_excel(writer, sheet_name='PPM_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
#print(bu_only_report_df)
#BEGINING OF COMPANY FILE

pmt_adj_bd_ppm_combined['Company']='Company_name'
pmt_adj_bd_combined_2['Company']='Company_name'
sub_ppm_df_copy['Company']='Company_name'

#agg groupby for COMPANY (sheet 1)--> Company_by_customer_type
Company_by_customer_type = pmt_adj_bd_ppm_combined.groupby(['Company','CustomerType'])\
    .agg({'ActiveCustomers':'sum','ActiveCustomers':'sum', 'Inactive':'sum', 'Total':'sum',  'EBMSConsumption':'sum', 'EnergyCharges':'sum',
          ' BilledAmount ': 'sum', 'Metered':'sum', 'Unmetered':'sum', 'Total_':'sum','DirectCustomers':'sum','AverageCustomers':'sum'
         ,'CustomersonMinimum':'sum','ReadCustomers':'sum', 'No_of_Bills':'sum','PmtAccountNumber':'count', 'PaidAmount':'sum', 'PreviousBalance':'sum'
         , 'OutstandingAmount':'sum', 'AmountEffected':'sum'}).round(2)
#print(Company_by_customer_type)
#agg groupby for COMPANY (sheet 2)--> Company_billing_summary_by_tariff
Company_billing_summary_by_tariff = pmt_adj_bd_combined_2.groupby(['Company', 'TariffDesc', 'TariffClassName'])\
    .agg({'ActiveCustomers':'sum',  'Inactive':'sum', 'Total': 'sum', 'EBMSConsumption':'sum', 'EnergyCharges':'sum', ' BilledAmount ': 'sum','Metered':'sum',
         'Unmetered':'sum','Total_':'sum', 'DirectCustomers':'sum','AverageCustomers':'sum','CustomersonMinimum':'sum','ReadCustomers':'sum', 'No_of_Bills':'sum', 
         'PmtAccountNumber':'count', 'PaidAmount':'sum', 'PreviousBalance':'sum', 'OutstandingAmount':'sum', 'AmountEffected':'sum', 'TariffRates': 'mean'}).round(2)
#print(Company_billing_summary_by_tariff)
#agg groupby for COMPANY (sheet 3)--> Company_ppm_billing_summary_by_tariff
Company_ppm_billing_summary_by_tariff = sub_ppm_df_copy.groupby(['Company', 'TariffDesc', 'TariffClassName'])\
    .agg({'ActiveCustomers':'sum',  'Inactive':'sum', 'Total': 'sum', 'EBMSConsumption':'sum', 'EnergyCharges':'sum', ' BilledAmount ': 'sum','Metered':'sum',
         'Unmetered':'sum','Total_':'sum', 'DirectCustomers':'sum','AverageCustomers':'sum','CustomersonMinimum':'sum','ReadCustomers':'sum', 'No_of_Bills':'sum', 
         'PPM_AccountNumber':'count', 'PaidAmount':'sum', 'PreviousBalance':'sum', 'OutstandingAmount':'sum', 'AmountEffected':'sum', 'TariffRates': 'mean'}).round(2)
#print(Company_ppm_billing_summary_by_tariff)
#RENAME Company COLUMN HEADERS
#Creating a sub for sheet 1
diso_consolidated_billing_summary_by_customer_type= Company_by_customer_type.reset_index()
diso_consolidated_billing_summary_by_customer_type.rename(columns=dict, inplace=True)

#Creating a sub for sheet 2
Company_postpaid_billing_summary_by_tariff= Company_billing_summary_by_tariff.reset_index()
Company_postpaid_billing_summary_by_tariff.rename(columns=dict, inplace=True)

#FOR SHEET 3
Company_cons_ppm_billing_summary_by_tariff= Company_ppm_billing_summary_by_tariff.reset_index()
Company_cons_ppm_billing_summary_by_tariff.rename(columns=dict, inplace=True)


# THIS GENERATES COMPANY CONSOLIDATED FILE
with pd.ExcelWriter('COMPANY Billing_Status.xlsx') as writer:  
            diso_consolidated_billing_summary_by_customer_type.to_excel(writer, sheet_name='BILLING_SUMMARY_BY_CUSTOMER_TYP', index=False, index_label='SNO')
            Company_postpaid_billing_summary_by_tariff.to_excel(writer, sheet_name='POST_BILLING_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
            Company_cons_ppm_billing_summary_by_tariff.to_excel(writer, sheet_name='PPM_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
#print(bu_only_report_df)
